In [1]:
reset()

# some auxiliary functions
def bits(n):
    return floor(log( max ( abs(n), 1) ,2))+1


def check_indexes(n,R,index,mode,R_bits):
    '''
    we make some checks concerning the constraints on n,R and mode
    ''' 
    if n%2==1:
        print("n must be even")
        return 1,0,0
    if R%mode != 0:
        print("R must be divisible by ",mode)
        return 1,_,_
    for x in R_bits[index-1]:
        #print(x)
        if x>1:
            if R%x != 0:
                print("R must be divisible by ",x)
                return 1,0,0
        else:
            continue  
    return 0,0,0


#///// Generation of the matrix and definition of the parameters #////  
    
def domains(index:int, R:int, R_bits:list):
    '''
    Constructs dynamically S_i according to predefined R-bits map (list), and the index selection
    Input
    -----
    index  : an index that picks a list from R_bits
    R      : bits
    R_bits : is a list of lists of the form [a,b], a,b integers with a<=b
    
    e.g.
    sage:R_bits = [[1,2], [2,4], [2,4,8], [1,2,4,8], [1,2,4,8,16]]
    sage:domains(2,10,R_bits)
    [(16, 31), (2, 3)] --> I_{R/2} x I_{R/4}
    
    sage:domains(1,10,R_bits)
    [(512, 1023), (16, 31)] --> I_R x I_{R/2}
    '''
    if index<1 or index>len(R_bits):
        return "index error"
    bounds = lambda R, den: (2 ** (R // den - 1), 2 ** (R // den) - 1)
    bisection = lambda R, dens: [bounds(R, d) for d in dens]
    bi = bisection(R, R_bits[index - 1]) # 0->1
    return bi 

def domain_selector(sector, n):
    '''
    This function is used to compute a random solution x of the system Ax=b where
    x in S. x has dimension n.
    Input:
    index   : is a positive integer
    sector  : is a list of lists [ [a1,a2],[b1,b2],...]
    n       : is a positive integer such that n = 0 mod len(sector)
    
    Output:
    returns a vector (w1,w2,...) where a1<=w1<=a2,....
    '''   
    if int(n)%int(len(sector)) !=0:
        print("n=",n," must be divisible by", len(sector))
        return 1,_,_
    a,_,_=check_indexes(n, R, index, mode, R_bits)
    if a==1:
        return 1,_,_
    #print(n,len(sector),n//len(sector),n/len(sector))
    rand = lambda n, bounds: [ZZ.random_element(bounds[0], bounds[1] + 1) for _ in range(n)]
    x = []
    for bounds in sector:
        x = x + rand(n//len(sector), bounds)
    return vector(x)



def gen_A_and_x(R, n, m, index, mode, R_bits):
    '''
    R : bits
    n : number of rows
    m : number of equations
    index = 1,2,3,4,5 or 6
    mode: denominator of \frac{R}{mode} that define the space I_{R/mode}, whose elements generate the matrix A.
    
    output : A, x, b, such that Ax=b
    '''    
    a,_,_=check_indexes(n, R, index, mode, R_bits)
    if a==1:
        return 1,_,_
    bits = lambda R, mode: (2 ** (R / mode - 1), 2 ** (R /mode) - 1)

    left, right = bits(R, mode) # To generate the matrix A

    # Now, we generate a matrix with (n-1) rows, and random integer elements from the interval [left, right]
    A = matrix([vector([ZZ.random_element(left, right + 1) for _ in range(n)]) for _ in range(m)])
    # Now, we choose a solution with constraints
    dom_sel = domain_selector(domains(index, R, R_bits), n)
    if dom_sel[0] == 1:
        return 1,_,_
    else:
        x = domain_selector(domains(index, R, R_bits), n)
    # and the constant vector of the system
    b = (A * x)  
    #print("b=",b)
    return A, x, b


#\\\\ the attack \\\\#

# we choose the target vector
def target(index:int, R:int, n:int, R_bits:list):
    '''
    Defines the target vector according to the selected S_{index}
    n : the dimension of the target vector
    '''
    def unpack_lists_tuples2(list:list)->list:
        return [i for sub in list for i in sub]
    
    if index<1 or index>len(R_bits):
        return "index error"
    t_r = lambda R, den: 2 ** (R // den - 1) + 2 ** (R // den - 2)
    bisection_tr = lambda R, dens, n: unpack_lists_tuples2([[t_r(R, d)] * (n // len(dens)) for d in dens])
    target_vector = vector(bisection_tr(R, R_bits[index - 1], n))
    return target_vector 

def kernel(A):
    Rank,k,r = rank_of_L(A)
    #print("rank of the lattice:",Rank )
    Q=A.smith_form()[2]
    X=matrix(ZZ,[Q.column(i) for i in range(r,k)])
    return X

def rank_of_L(A):
    '''
    outputs three integers. 
    [1] the rank of the lattice Ax=0, which is equal to (number of columns of A)-(rank(A)), 
    This is in fact the dimension of the nullspace(A).
    [2] the number of columns k and 
    [3] the rank r of A
    '''
    k=A.dimensions()[1] # number of columns
    r=A.rank()
    return k-r,k,r

def babai(B,v):  # TODO: Replace with the one of Fpylll
    Y=[]
    i=0
    j=0
    row=int(B.nrows())
    col=int(B.ncols())
    w=vector([0 for i in range(0,len(v))])
    Gram=B.gram_schmidt()[0] # Gram_schmidt is a function of Sagemath
    w=vector(v)
    for j in range(row):        
        i=row-j-1
        c1=w.dot_product((Gram[i])) # dot product is a function of Sagemath
        c2=Gram[i].dot_product(Gram[i])
        l=c1/c2
        e=floor(l+0.5)*(B.row(i))
        Y.append(e)
        w=w-(l-floor(l+0.5))*Gram[i]-(floor(l+0.5)*B.row(i))   
    u=sum(Y)        
    return u


def run_babai(index, R, A, R_bits, target):
    basis = kernel(A)
    if rank_of_L(A)[0] > 1:  # we want at least two rows to apply LLL reduction to matrix A
        B_lll = basis.LLL()  # before we execute Babai we reduce the basis with LLL or BKZ
    else:
        B_lll = basis
    # Alll=A.BKZ(block_size=35) # uncomment if you want to use BKZ
    n = A.ncols()
    #t = target(index, R, n,  R_bits)
    # print(f'Target: {t}')

    # We execute the attack
    # First we execute approximate CVP using Babai's algorithm
    bab = babai(B_lll, target)
    return bab 

def a_solution_of_the_system(A,d,N1,N2):
    def construction_of_matrix(A,d,N1,N2):
        '''
        input   : dimension n, a matrix mxn A = [aij], a column vector d
                  and two parameters N1, N2 with N1<N2

        output  : a solution of the system Ax=d
        '''

        n,m=A.dimensions()[1] , A.dimensions()[0] # we define the rows (m) and columns (n)    
        # construction of the matrix
        zerom1 = matrix(QQ,1,[0 for i in range(0, n)]).T;     # 0_{nx1}
        zerom2 = matrix(QQ,1,[0 for i in range(0, n)]+[N1]);  # [0_{1xn},N1]
        In = identity_matrix(ZZ, n);
        upper_block = block_matrix([[In,zerom1]]);
        medium_row = zerom2
        lower_block = block_matrix([[N2*A,-N2*d]]);
        final_matrix = block_matrix(3,[ [upper_block], [zerom2], [lower_block]])
        return final_matrix.T
    
    #todo : put a verification
    if A==_:
        return
    n = A.dimensions()[1] # was referred without being defined
    B=construction_of_matrix(A,d,N1,N2)
    Blll = B.LLL()
    #print(Blll)
    
    #in order to find the solutions, you have to find the row that has the entry N1 
    #i.e. we are looking for a row of the form (....,N1,...)
    
    nrows = Blll.dimensions()[0]
    for i in range(nrows):
        if N1 in Blll.row(i):
            t = i
            solution = Blll.row(t)[:n]
            #print("verification for a partial solution",A*matrix(solution).T==matrix(d))
            #print("solution:",solution)
            return solution
    print("error: We could not find a partial solution. Probably, because of the choice of N1,N2.")
    return "error"


#Modified functions for dynamic execution
def number_of_good_entries_single(list, bound):  # the number of entries in the interval [lower,upper]
    '''
    sage:number_of_good_entries_single([2,10,11,90],[90,100])
         (1,3)
    '''
    C = 0
    inbound = lambda x, low, up: (x <= up and x >= low)
    for i in range(len(list)):
        if inbound(list[i], *bound):
            C = C + 1
    return C, len(list) - C  # number of good and bad entries

def solution_verification_constraints(alpha:int, index:int, Xd:list, R:int, R_bits:list, prime_sector:list):
    number_of_good_entries = number_of_good_entries_single
    M = []
    #print("prime_sector",prime_sector)
    if alpha != 0:
        for i in range(2 * alpha):
            temp = number_of_good_entries(list(Xd[i]), prime_sector)
            C = temp[0]
            M.append(C)
    else:
        temp = number_of_good_entries(list(Xd), prime_sector)
        C = temp[0]
        return C
    return max(M)

def Xd_list(alpha,babai_vector,sol,n): #Edit: just add n as argument
    '''
    input  : an integer alpha, usually 10 and the output of babai(.,.)
             sol is a solution of the non homogeneouos
             n is the dimension (number of columns of A)
    output : a list that contains the vectors, sol +j*babai_vector, for j=-alpha,...,alpha 
    '''
    Xdn = [] # Xd negative
    Xdp = [] # Xd positive
    Xd  = [] # the union of Xdn and Xdp
    if babai_vector!=vector([0]*n):
        Xdn = [vector(sol) - j*vector(babai_vector) for j in range(alpha)] 
        Xdp = [vector(sol) + j*vector(babai_vector) for j in range(alpha)] 
        Xd = Xdn+Xdp
    else: # i.e. in the case where babai provides the zero vector
        Xd = vector(sol)
        alpha = 0
    #print("lenXd:",len(Xd),alpha)
    return Xd,alpha


def second_attack_parameters(R, n, m, index, mode, R_bits):
    '''
    idea : split A to (Ai)
    split target vector t to (ti)
    generate local constants (cons[i])
    b remains the same  (b)
    split the solutions: i.e. solve each new Ai*X = cons[i]
    '''
    # [1]  define the parameters
 
    Ai=[]
    sectors = domains(index,R,R_bits)
    len_sectors = len(sectors)
    # [2] split the matrix
    A,x,b = gen_A_and_x(R, n, m, index, mode, R_bits)
    #print(A*x==b)
    auxiliary_list = [n*i/len_sectors for i in range(len_sectors)]
    Ai.append([matrix([A.column(i) for i in range(j,n/len_sectors + j)]).T for j in auxiliary_list])
    Ai = Ai[0]
    # [3] split the target vector
    t =  target(index, R, n, R_bits)
    ti = [t[0+(i-1)*n/len_sectors:i*n/len_sectors] for i in range(1,len_sectors+1)]
    # [4] generate constants. Here we choose bi=ai , i>=2
    # TODO :  provide b2,...bk as input
    nonce=[]    
    nonce.append([ [ZZ.random_element(sectors[i][0],sectors[i][1]) for j in range(m)] \
                  for i in range(1,len_sectors) ])
    nonce=nonce[0]
    #print("nonce",nonce)
    cons = [b - sum(vector(nonce[i]) for i in range(len(nonce)))] + nonce
        
    # [5] Generate local solutions
    # xi is a list of solutions
    N1, N2 = int(2 ** R), int(2 ** (R + 2))
    #Ν1,Ν2=int(1e10),int(1e30)
    xi=[[a_solution_of_the_system(Ai[i],matrix(cons[i]).T,N1,N2)][0] for i in range(len_sectors)]   

    return Ai,xi,cons,ti,len_sectors,nonce,b,sectors


def attack(A, sol, ti, index, R, R_bits, alpha, prime_sector):
    
    # We execute the attack
    # First we execute approximate CVP using Babai's algorithm
    M = [] # will keep the solutions in this set
    n = A.ncols()
    m = A.nrows()
    babai_vector = run_babai(index, R, A, R_bits,ti)
    Xd, alpha = Xd_list(alpha, babai_vector, sol,n)
    out = solution_verification_constraints(alpha, index, Xd, R, R_bits,prime_sector)
    return out,Xd


def attack_(Ai, xi, ti, index, R, R_bits, alpha, sectors):
    s = [attack(Ai[i],xi[i],ti[i],index,R,R_bits,alpha,sectors[i]) for i in range(Len)]
    #print(s)
    ss  = [x[1] for x in s] # these are the solutions
    sss = [x[0] for x in s] # these are non negative integers
    return ss,sss 

def stats(mode, n, m, R_bits, count, H):
    import numpy as np
    print("R=", R)
    print("bits of A=(aij)=", "R /", mode)
    print("we picked R_bits to be:",R_bits[index-1])
    print("unknowns=", n)
    print("equations=", m)
    print("number of instances considered:", count)
    print("average number of good hits:", "{:.2f}".format(round(np.mean(H), 2)), ",", "percentage :",
          float(100 * np.mean(H) / n))
    print("the minumum of succeded hits:", min(H))
    print("the maximum :", max(H))
    if max(H) == n:
        print("+++ We found at least one solution +++")

In [2]:
R_bits = [[1,2], [2,4], [2,4,8], [1,2,4,8], [1,2,4,8,16],[1,2,4,8,16,32],[1,2,4,8,16,32,64]]
index = 4 # starting from 1 not 0
R = 80
n = 48
m = 10
mode = 8
count = 20

In [3]:
alpha = 40
H = []
for i in range(count):
    Ai,xi,cons,ti,Len,nonce,b,sectors=second_attack_parameters(R, n, m, index, mode, R_bits)
    att,O = attack_(Ai, xi, ti, index, R, R_bits, alpha, sectors)
    H.append(sum(O))
stats(mode, n, m, R_bits, 10, H)

R= 80
bits of A=(aij)= R / 8
we picked R_bits to be: [1, 2, 4, 8]
unknowns= 48
equations= 10
number of instances considered: 10
average number of good hits: 12.95 , percentage : 26.979166666666668
the minumum of succeded hits: 12
the maximum : 16
